Import Statements

In [44]:
import cv2
import time
import numpy as np
import multiprocessing as mp
import os
import errno
from subprocess import Popen, PIPE, CalledProcessError
import sys, subprocess
import shutil
from pathlib import Path
import regex as re
import tqdm
import io
from base64 import b64encode
from IPython.display import HTML
from google.colab import files

Clone Repository for Monodepth2 and YoloP code

In [2]:
if os.path.exists('proj-mobility'):
  shutil.rmtree('proj-mobility')
!git clone https://github.com/sashanksridhar/proj-mobility

Cloning into 'proj-mobility'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 164 (delta 2), reused 11 (delta 2), pack-reused 148
Receiving objects: 100% (164/164), 138.14 MiB | 36.03 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Checking out files: 100% (120/120), done.


Install Requirements

In [3]:
!pip install -r /content/proj-mobility/YOLOP/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 6.8 MB/s 
     |████████████████████████████████| 125 kB 66.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Upload Files (Preferably .mp4 as the following codes will have to be edited for other types)

Can upload multiple files at once (Select one by one (or) all together)

In [4]:
uploaded = files.upload()

Saving 1.mp4 to 1.mp4


Check for Video files in .mp4 format

In [5]:
mp4_files = [f for f in os.listdir('./') if f.endswith('.mp4')]
print(mp4_files)

['1.mp4']


For parallel processing of frames get the number of cpus

In [6]:
num_processes = mp.cpu_count()

Function to extract frames

In [7]:
def process_video(args):
    print(args)
    input_file, group_number, oppath = args[0], args[1], args[2]
    cap = cv2.VideoCapture(input_file)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_jump_unit * group_number)
    proc_frames = 0

    while proc_frames < frame_jump_unit:
        ret, frame = cap.read()
        if ret == False:
            break
        # frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        cv2.imwrite(oppath + "/%s_y.jpg" % str(frame_jump_unit * group_number + proc_frames), frame)
        proc_frames += 1


Extract and write frames to a 'frames' directory

In [8]:
if os.path.exists('frames'):
  shutil.rmtree('frames')

try:
  os.makedirs('frames')
except OSError as e:
    if e.errno == errno.EEXIST:
      raise
  

for mp4 in mp4_files:
  name = os.path.splitext(mp4)[0]
  os.makedirs(os.path.join('frames', name))
  cap = cv2.VideoCapture(mp4)
  frame_jump_unit = cap.get(cv2.CAP_PROP_FRAME_COUNT) // num_processes
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  print(fps)
  kernel = np.ones((7, 7), np.float32) / 49
  start_time = time.time()
  p = mp.Pool(num_processes)
  for i in range(num_processes):
    p.map_async(process_video, [[mp4, i ,os.path.join('frames', name)]])

  t3 = time.time()

  print(t3 - start_time)

267.0
30.0
0.03142046928405762
['1.mp4', 0, 'frames/1']['1.mp4', 1, 'frames/1']


Process both YoloP and Monodepth2 Inferences.

Output logs might take time to reflect as it is a subprocess running in the background.

In [9]:
for folder in next(os.walk('frames'))[1]:
  if folder == [] or folder.startswith("."):
    continue
  print(folder)

  if os.path.exists(os.path.join('frames',folder+"_yolo_output")):
    shutil.rmtree(os.path.join('frames',folder+"_yolo_output"))
  if os.path.exists(os.path.join('frames',folder+"_mono_output")):
    shutil.rmtree(os.path.join('frames',folder+"_mono_output"))
  
  if "_output" in folder:
    continue
  
  os.makedirs(os.path.join('frames',folder+"_yolo_output"))
  os.makedirs(os.path.join('frames',folder+"_mono_output"))
  jpg = [f for f in os.listdir(os.path.join('frames',folder))]
  
  cmd = "python /content/proj-mobility/YOLOP/tools/demo.py --source '"+os.path.join("frames", folder) +"' --save-dir '"+ os.path.join('frames',folder+"_yolo_output")+"' --weights '/content/proj-mobility/YOLOP/weights/End-to-end.pth' --device 0,1 "
  process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

  while True:
      nextline = process.stdout.readline()
      if  nextline == bytes('', encoding='utf-8'):
          break
      sys.stdout.write(nextline)
      sys.stdout.flush()

  output = process.communicate()[0]
  exitCode = process.returncode

  cmd = "python /content/proj-mobility/monodepth2/test_simple.py --image_path '"+os.path.join("frames", folder) +"' --save_dir '"+ os.path.join('frames',folder+"_mono_output")+"' --model_name mono_640x192"
  process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

  while True:
      nextline = process.stdout.readline()
      if nextline == bytes('', encoding='utf-8'):
          break
      sys.stdout.write(nextline)
      sys.stdout.flush()

  output = process.communicate()[0]
  exitCode = process.returncode


1
Using torch 1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

['/content/proj-mobility/YOLOP/tools', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/content/proj-mobility/YOLOP']
=> creating runs/BddDataset/_2022-08-17-09-00
/content/proj-mobility/YOLOP/weights/End-to-end.pth
  0%|          | 0/231 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 78%|███████▊  | 181/231 [00:43<00:11,  4.44it/s]image 1/231 /content/frames/1/0.0_y.jpg: 
image 2/231 /content/frames/1/1.0_y.jpg: 
image 3/231 /content/frames/1/10.0_y.jpg: 
image 4/231 /content/frames/1/100.0_y.jpg: 
image 5/231 /content/

In [ ]:
# !python /content/proj-mobility/YOLOP/tools/demo.py --source '/content/frames/YoloPInput1' --save-dir '/content/frames/YoloPInput1_output' --weights '/content/proj-mobility/YOLOP/weights/End-to-end.pth' --device 0,1

In [ ]:
# !python /content/proj-mobility/monodepth2/test_simple.py --image_path /content/frames/YoloPInput1 --model_name mono_640x192 --save_dir /content/frames/YoloPInput1_mono_output

Creating Output Video and Downloading

In [45]:
for folder in next(os.walk('frames'))[1]:
  if folder == [] or folder.startswith("."):
    continue
  print(folder)
  
   
  if "_output" in folder:
    continue
  
  if not (os.path.exists(os.path.join('frames',folder+"_mono_output")) and os.path.exists(os.path.join('frames',folder+"_yolo_output"))):
    continue

  yolo = [f for f in os.listdir(os.path.join('frames',folder+"_yolo_output"))]

  mono = [f for f in os.listdir(os.path.join('frames',folder+"_mono_output")) if ".jpeg" in f]

  
  fps_w = 30
  size_w = (2 * 1280, 2 * 720)
  writer = cv2.VideoWriter(folder+'_out.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps_w, size_w)


  def num_sort(frame):
    return list(map(int, re.findall(r'\d+', frame)))[0]


  yolo.sort(key=num_sort)
  mono.sort(key=num_sort)
  print(yolo)
  print(mono)

  for i in tqdm.tqdm(range(len(yolo))):
    filename1 = os.path.join('frames',folder+"_yolo_output", yolo[i])
    frame1 = cv2.imread(filename1)

    filename2 = os.path.join('frames',folder+"_mono_output", mono[i])
    frame2 = cv2.imread(filename2)

    frame2 = cv2.resize(frame2, (frame1.shape[1], frame1.shape[0]), interpolation=cv2.INTER_AREA)


    vis = np.concatenate((frame1, frame2), axis=1)

    height, width, layers = vis.shape
    frame = cv2.resize(vis, size_w, interpolation=cv2.INTER_AREA)
    writer.write(frame)


  writer.release()

  files.download(folder+'_out.mp4')

 

1
['0.0_y.jpg', '1.0_y.jpg', '2.0_y.jpg', '3.0_y.jpg', '4.0_y.jpg', '5.0_y.jpg', '6.0_y.jpg', '7.0_y.jpg', '8.0_y.jpg', '9.0_y.jpg', '10.0_y.jpg', '11.0_y.jpg', '12.0_y.jpg', '13.0_y.jpg', '14.0_y.jpg', '15.0_y.jpg', '16.0_y.jpg', '17.0_y.jpg', '18.0_y.jpg', '19.0_y.jpg', '20.0_y.jpg', '21.0_y.jpg', '22.0_y.jpg', '23.0_y.jpg', '24.0_y.jpg', '25.0_y.jpg', '26.0_y.jpg', '27.0_y.jpg', '28.0_y.jpg', '29.0_y.jpg', '30.0_y.jpg', '31.0_y.jpg', '32.0_y.jpg', '33.0_y.jpg', '34.0_y.jpg', '35.0_y.jpg', '36.0_y.jpg', '37.0_y.jpg', '38.0_y.jpg', '39.0_y.jpg', '40.0_y.jpg', '41.0_y.jpg', '42.0_y.jpg', '43.0_y.jpg', '44.0_y.jpg', '45.0_y.jpg', '46.0_y.jpg', '47.0_y.jpg', '48.0_y.jpg', '49.0_y.jpg', '50.0_y.jpg', '51.0_y.jpg', '52.0_y.jpg', '53.0_y.jpg', '54.0_y.jpg', '55.0_y.jpg', '56.0_y.jpg', '57.0_y.jpg', '58.0_y.jpg', '59.0_y.jpg', '60.0_y.jpg', '61.0_y.jpg', '62.0_y.jpg', '63.0_y.jpg', '64.0_y.jpg', '65.0_y.jpg', '66.0_y.jpg', '67.0_y.jpg', '68.0_y.jpg', '69.0_y.jpg', '70.0_y.jpg', '71.0_y.jpg',

100%|██████████| 231/231 [00:22<00:00, 10.16it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1_yolo_output
1_mono_output


To display the video on colab. Replace 'filename' with the video name

In [ ]:
# !ffmpeg -i 1_out.mp4 -c:v libvpx -preset ultrafast 1_10s.webm

In [46]:
filename = '1_out.mp4'
cmd = "ffmpeg -i "+ filename +" -c:v libvpx -preset ultrafast "+ filename+"_web.webm"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

while True:
    nextline = process.stdout.readline()
    if nextline == bytes('', encoding='utf-8'):
        break
    sys.stdout.write(nextline)
    sys.stdout.flush()

output = process.communicate()[0]
exitCode = process.returncode

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

Output Video

In [47]:
with  io.open(filename+'_web.webm','r+b') as f:
    mp4 = f.read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/webm">
</video>
""" % data_url)